In [37]:
import pandas as pd
from typing import List

from analysis.utils.cleaning import lower_case_and_strip_spaces

### The Data and what we want to do with it:

These datasets where originally from Kaggle here is the description they gave:
This dataset contains 27753444 ratings across 58098 movies. These data sets were created by 283228 users between January 09, 1995 and September 26, 2018. This dataset was generated on September 26, 2018.

### Reading the Data and Initial Stats

In [4]:
movies_df: pd.DataFrame = pd.read_csv('input/all_movies.csv')
movies_df.sample(20)

,movieId,title,genres
36737,145222,Bedlam (2015),Thriller
27708,123254,State Fair (1962),(no genres listed)
55004,186405,Ben 10 - Alien Swarm (2009),Adventure
27385,122533,Breach of Trust (1995),Action|drama|THRILLER
23402,109935,The Artist and the Model (2012),Drama
2295,2379,Police Academy 2: Their First Assignment (1985),Comedy|crime
50359,176165,Cop and a Half: New Recruit (2017),Action|children|COMEDY|crime
18160,90405,In Time (2011),Crime|sci-fi|THRILLER
39854,152894,Lakeer - Forbidden Lines (2004),Action|drama|ROMANCE
49275,173771,McLaren (2016),Action|documentary|DRAMA


In [6]:
movies_df

,movieId,title,genres
0,1,Toy Story (1995),Adventure|animation|CHILDREN|comedy|Fantasy
1,2,Jumanji (1995),Adventure|children|FANTASY
2,3,Grumpier Old Men (1995),Comedy|romance
3,4,Waiting to Exhale (1995),Comedy|drama|ROMANCE
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
58093,193876,The Great Glinka (1946),(no genres listed)
58094,193878,Les tribulations d'une caissière (2011),Comedy
58095,193880,Her Name Was Mumu (2016),Drama
58096,193882,Flora (2017),Adventure|drama|HORROR|sci-fi


In [7]:
ratings_df = pd.read_csv('input/ratings.csv')
ratings_df.sample(20)

,userId,movieId,rating,timestamp
8102165,83406,1234,5.0,1095728493
12400877,127058,367,2.0,1111287555
17038902,174150,608,5.0,945700940
2669324,27508,357,3.0,846323768
23571102,240828,40826,2.5,1136767320
27431524,279863,3608,3.0,1366525194
20661989,211008,529,5.0,845075838
11705337,120061,65,3.0,1152720027
17019370,173990,63859,3.5,1272740060
18222198,185980,43897,3.5,1299948170


## Step 1: Cleaning
You shouldn't necessarily assume that your data is good.  It could be very sparse and not have much there. There could be duplication, poorly recorded or empty values, or with large text there could be a lot of garbage in there if it was an open text field

Some things we'll do here:
1. Get the number of rows and columns by looking at the shape
2. Determine the number of non-null rows for each given column (This would tell us if a column is especially sparse)
3. Check for duplicate rows

As we find things that need to be cleaned (bad text, duplicates etc.) we will write tested cleaning functions to cleanup our input data.

In [8]:
movies_df.shape

(58098, 3)

In [9]:
movies_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58098 entries, 0 to 58097
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   movieId  58098 non-null  int64 
 1   title    58098 non-null  object
 2   genres   58098 non-null  object
dtypes: int64(1), object(2)
memory usage: 1.3+ MB


#### Checking for Duplicates

First we need to define what a "duplicate" is.

In [10]:
is_title_duplicated = movies_df.duplicated(['title'])

In [12]:
movie_where_title_is_duplicated_df = movies_df.loc[is_title_duplicated]

In [13]:
movie_where_title_is_duplicated_df.shape

(78, 3)

In [14]:
is_title_and_genre_duplicated = movies_df.duplicated(['title', 'genres'])

In [15]:
movie_where_title_and_genre_is_duplicated_df = movies_df.loc[is_title_and_genre_duplicated]

In [16]:
movie_where_title_and_genre_is_duplicated_df.shape

(14, 3)

Let's do some cleanup.

In [38]:
movies_df['genres'] = movies_df['genres'].apply(lambda x: lower_case_and_strip_spaces(x))

In [40]:
movies_df = movies_df.loc[movies_df['genres'] != '(no genres listed)']

In [41]:
movies_df.shape

(53832, 3)

In [42]:
movie_where_title_is_duplicated_df = movies_df.loc[is_title_duplicated]
movie_where_title_is_duplicated_df.shape

(70, 3)

In [45]:
movie_where_title_and_genre_is_duplicated_df = movies_df.loc[is_title_and_genre_duplicated]
movie_where_title_and_genre_is_duplicated_df.shape

(14, 3)

Now let's determine what is happening to cause the shape discrepency

In [46]:
duplicates_by_title = set(movie_where_title_is_duplicated_df['title'])
duplicates_by_title

{'20,000 Leagues Under the Sea (1997)',
 'Absolution (2015)',
 'Aftermath (2012)',
 'Aladdin (1992)',
 'Beneath (2013)',
 'Berlin Calling (2008)',
 'Blackout (2007)',
 'Cargo (2017)',
 'Casanova (2005)',
 'Chaos (2005)',
 'Classmates (2016)',
 'Clear History (2013)',
 'Clockstoppers (2002)',
 'Confessions of a Dangerous Mind (2002)',
 'Contact (1992)',
 'Darling (2007)',
 'Delirium (2014)',
 'Deranged (2012)',
 'Detour (2017)',
 'Dracula (1931)',
 'Ecstasy (2011)',
 'Eden (2014)',
 'Emma (1996)',
 'Eros (2004)',
 'Escape Room (2017)',
 'Forsaken (2016)',
 'Free Fall (2014)',
 'Frozen (2010)',
 'Girl, The (2012)',
 'Good People (2014)',
 'Gossip (2000)',
 'Grace (2014)',
 'Hamlet (2000)',
 'Holiday (2014)',
 'Home (2008)',
 'Hostage (2005)',
 'Inside (2012)',
 'Interrogation (2016)',
 'Johnny Express (2014)',
 'Journey to the Center of the Earth (2008)',
 'Lagaan: Once Upon a Time in India (2001)',
 'Let There Be Light (2017)',
 'Little Man (2006)',
 'Lucky (2017)',
 'Macbeth (2015)',
 

In [48]:
duplicates_by_title_and_genre = set(movie_where_title_and_genre_is_duplicated_df['title'])
duplicates_by_title_and_genre

{'Beneath (2013)',
 'Berlin Calling (2008)',
 'Clear History (2013)',
 'Darling (2007)',
 'Detour (2017)',
 'Dracula (1931)',
 'Girl, The (2012)',
 'Home (2008)',
 'Johnny Express (2014)',
 'Little Man (2006)',
 'Lucky (2017)',
 'Macbeth (2015)',
 'Offside (2006)',
 'Seven Years Bad Luck (1921)'}

Here we have a situation where 56 things are duplicated based on their title but the two rows have different genres.  How can we easily find an example movie where this is the case?

In [52]:
movie_where_title_is_duplicated_but_not_genre = duplicates_by_title.difference(duplicates_by_title_and_genre)
list(movie_where_title_is_duplicated_but_not_genre)

['Emma (1996)',
 'Chaos (2005)',
 'Ecstasy (2011)',
 'Let There Be Light (2017)',
 'Office (2015)',
 'Noise (2007)',
 'Contact (1992)',
 'Hamlet (2000)',
 'Stranded (2015)',
 'Lagaan: Once Upon a Time in India (2001)',
 'The Promise (2016)',
 'The Connection (2014)',
 'Men with Guns (1997)',
 'Inside (2012)',
 'Weekend (2011)',
 'Absolution (2015)',
 'Deranged (2012)',
 'The Midnight Man (2016)',
 'Interrogation (2016)',
 'Shelter (2015)',
 'The Void (2016)',
 'Holiday (2014)',
 'The Break-In (2016)',
 'Good People (2014)',
 'Aftermath (2012)',
 'Grace (2014)',
 'Truth (2015)',
 'Journey to the Center of the Earth (2008)',
 'Classmates (2016)',
 'Clockstoppers (2002)',
 'Eden (2014)',
 'Forsaken (2016)',
 'Hostage (2005)',
 'Paradise (2013)',
 'Saturn 3 (1980)',
 'Sing (2016)',
 'The Dream Team (2012)',
 'Gossip (2000)',
 'Cargo (2017)',
 'Rose (2011)',
 'Eros (2004)',
 'Blackout (2007)',
 'Confessions of a Dangerous Mind (2002)',
 'Frozen (2010)',
 'Veronica (2017)',
 'Casanova (2005)

Let's check out an example:

In [53]:
movies_df.loc[movies_df['title'] == 'Aladdin (1992)']

,movieId,title,genres
582,588,Aladdin (1992),adventure|animation|children|comedy|musical
24657,114240,Aladdin (1992),adventure|animation|children|comedy|fantasy


In [54]:
movies_df.loc[movies_df['title'] == 'Free Fall (2014)']

,movieId,title,genres
44865,164300,Free Fall (2014),drama
46947,168866,Free Fall (2014),action|drama|thriller


In [56]:
from utils.cleaning import find_duplicates_and_combine

movies_with_combined_genres_df = find_duplicates_and_combine(movies_df, list(movie_where_title_is_duplicated_but_not_genre))

/Users/lizjohnson/projects/simple_recommendation_engine/venv/lib/python3.7/site-packages/pandas/core/indexing.py:1884: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, val, pi)


In [57]:
assert len(movies_with_combined_genres_df.index) == len(movies_df.index)

In [58]:
movies_with_combined_genres_df.loc[movies_with_combined_genres_df['title'] == 'Aladdin (1992)']

,movieId,title,genres
582,588,Aladdin (1992),adventure|animation|children|comedy|musical|fa...
24657,114240,Aladdin (1992),adventure|animation|children|comedy|musical|fa...


In [69]:
movie_where_title_and_genre_is_duplicated_df = movies_with_combined_genres_df.loc[is_title_and_genre_duplicated]
movie_where_title_and_genre_is_duplicated_df.shape

(14, 3)

### Ratings Data Frame
Same Steps
1. Easily get the number of rows and columns by looking at the shape
2. Determine the number of non-null rows for each given column
3. Check for duplicate rows

In [59]:
ratings_df.shape

(27753444, 4)

In [60]:
ratings_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27753444 entries, 0 to 27753443
Data columns (total 4 columns):
 #   Column     Dtype  
---  ------     -----  
 0   userId     int64  
 1   movieId    int64  
 2   rating     float64
 3   timestamp  int64  
dtypes: float64(1), int64(3)
memory usage: 847.0 MB


Here we should again define what it means to be a "duplicate":

In [63]:
assert (len(ratings_df) - len(ratings_df.drop_duplicates(['userId', 'movieId', 'rating', 'timestamp']))) == 0

## Step 2: Feature Preparation
We want to be able to recommend movies to users. What are some things we could do next?

Answer:
- Join movies to ratings
- Group by movie title to find an average rating for each movie

Here we will left join movies to their ratings.  If we expect all movies to have ratings than we can start with an assertion that indicates that.

In [64]:
movies_and_ratings_df = movies_df.set_index('movieId').join(ratings_df.set_index('movieId')).reset_index()

In [65]:
movies_and_ratings_df

,movieId,title,genres,userId,rating,timestamp
0,1,Toy Story (1995),adventure|animation|children|comedy|fantasy,4.0,4.0,1.113766e+09
1,1,Toy Story (1995),adventure|animation|children|comedy|fantasy,10.0,5.0,9.488858e+08
2,1,Toy Story (1995),adventure|animation|children|comedy|fantasy,14.0,4.5,1.442169e+09
3,1,Toy Story (1995),adventure|animation|children|comedy|fantasy,15.0,4.0,1.370810e+09
4,1,Toy Story (1995),adventure|animation|children|comedy|fantasy,22.0,4.0,1.237623e+09
...,...,...,...,...,...,...
27738725,193878,Les tribulations d'une caissière (2011),comedy,176871.0,2.0,1.537875e+09
27738726,193880,Her Name Was Mumu (2016),drama,81710.0,2.0,1.537886e+09
27738727,193882,Flora (2017),adventure|drama|horror|sci-fi,33330.0,2.0,1.537891e+09
27738728,193886,Leal (2018),action|crime|drama,206009.0,2.5,1.537918e+09


Let's find the movies that were missing ratings (if any).

In [67]:
movies_missing_ratings = movies_and_ratings_df.loc[movies_and_ratings_df['rating'].isna()]
movies_missing_ratings

,movieId,title,genres,userId,rating,timestamp
21871974,25817,Break of Hearts (1935),drama|romance,NaN,NaN,NaN
21914110,26361,Baby Blue Marine (1976),drama,NaN,NaN,NaN
21971501,27153,Can't Be Heaven (Forever Together) (2000),children|comedy|drama|romance,NaN,NaN,NaN
21986044,27433,Bark! (2002),comedy|drama,NaN,NaN,NaN
22225201,31945,Always a Bridesmaid (2000),documentary,NaN,NaN,NaN
...,...,...,...,...,...,...
27737891,192399,Under Wraps (1997),children|comedy|horror,NaN,NaN,NaN
27738267,192933,Rosie (2018),drama,NaN,NaN,NaN
27738335,193109,Ach śpij kochanie (2017),crime|thriller,NaN,NaN,NaN
27738455,193321,Pledges (2018),comedy|horror,NaN,NaN,NaN


Now let's keep only movies that did have ratings

In [68]:
movies_with_ratings_df = movies_and_ratings_df.loc[movies_and_ratings_df['rating'].notna()]
movies_with_ratings_df

,movieId,title,genres,userId,rating,timestamp
0,1,Toy Story (1995),adventure|animation|children|comedy|fantasy,4.0,4.0,1.113766e+09
1,1,Toy Story (1995),adventure|animation|children|comedy|fantasy,10.0,5.0,9.488858e+08
2,1,Toy Story (1995),adventure|animation|children|comedy|fantasy,14.0,4.5,1.442169e+09
3,1,Toy Story (1995),adventure|animation|children|comedy|fantasy,15.0,4.0,1.370810e+09
4,1,Toy Story (1995),adventure|animation|children|comedy|fantasy,22.0,4.0,1.237623e+09
...,...,...,...,...,...,...
27738725,193878,Les tribulations d'une caissière (2011),comedy,176871.0,2.0,1.537875e+09
27738726,193880,Her Name Was Mumu (2016),drama,81710.0,2.0,1.537886e+09
27738727,193882,Flora (2017),adventure|drama|horror|sci-fi,33330.0,2.0,1.537891e+09
27738728,193886,Leal (2018),action|crime|drama,206009.0,2.5,1.537918e+09


In [72]:
movies_with_ratings_combined_df = movies_with_ratings_df.groupby('title').agg({'rating': lambda x: x.to_list(), 'genres': lambda x: x.to_list()[0]}).reset_index()

In [73]:
movies_with_ratings_combined_df

,title,rating,genres
0,"""Great Performances"" Cats (1998)","[3.0, 5.0, 4.0, 1.0, 0.5, 1.5, 0.5, 3.5, 5.0, ...",musical
1,#1 Cheerleader Camp (2010),"[3.0, 1.0, 1.5, 3.0, 4.0, 2.0, 4.0, 5.0, 1.5]",comedy|drama
2,#Captured (2017),[2.5],horror
3,#Horror (2015),"[1.0, 1.0, 3.0, 3.5, 0.5, 2.5, 1.5, 2.0, 3.0, ...",drama|horror|mystery|thriller
4,#SCREAMERS (2016),[2.5],horror
...,...,...,...
50091,ארבינקא (1967),[3.5],comedy|crime|romance
50092,…And the Fifth Horseman Is Fear (1965),"[3.5, 3.0]",drama|war
50093,キサラギ (2007),"[3.5, 4.5, 3.0]",comedy|mystery
50094,チェブラーシカ (2010),"[5.0, 4.0, 2.5, 3.5, 3.5, 4.5, 3.0, 3.0, 2.0, ...",animation|children


Next let's get an average rating for each movie by grouping by title and creating a list of the ratings and taking the first instance of a genres list.

In [77]:
def average_rating(ratings: List) -> float:
    return sum(ratings)/len(ratings)

In [78]:
movies_with_ratings_combined_df['average_rating'] = movies_with_ratings_combined_df['rating'].apply(average_rating)

In [79]:
movies_with_ratings_combined_df

,title,rating,genres,average_rating
0,"""Great Performances"" Cats (1998)","[3.0, 5.0, 4.0, 1.0, 0.5, 1.5, 0.5, 3.5, 5.0, ...",musical,2.815556
1,#1 Cheerleader Camp (2010),"[3.0, 1.0, 1.5, 3.0, 4.0, 2.0, 4.0, 5.0, 1.5]",comedy|drama,2.777778
2,#Captured (2017),[2.5],horror,2.500000
3,#Horror (2015),"[1.0, 1.0, 3.0, 3.5, 0.5, 2.5, 1.5, 2.0, 3.0, ...",drama|horror|mystery|thriller,2.078947
4,#SCREAMERS (2016),[2.5],horror,2.500000
...,...,...,...,...
50091,ארבינקא (1967),[3.5],comedy|crime|romance,3.500000
50092,…And the Fifth Horseman Is Fear (1965),"[3.5, 3.0]",drama|war,3.250000
50093,キサラギ (2007),"[3.5, 4.5, 3.0]",comedy|mystery,3.666667
50094,チェブラーシカ (2010),"[5.0, 4.0, 2.5, 3.5, 3.5, 4.5, 3.0, 3.0, 2.0, ...",animation|children,3.150000


Let's think about our recommendation engine now again.  Let's say that we want to recommend movies to by recommending the movies with the most similar genres list.

In order to to use TF IDF we need a list of all the "words" (genres) used in our corpus.  This is easy for us to do.  We can make a list of all the genres by:
1. Creating a column with a list of genres
2. Grouping by the genre
3. Aggregating the results
4. Transforming the resulting series into a list of genres

Column with list of genres

In [80]:
movies_with_ratings_combined_df['genres'] = movies_with_ratings_combined_df['genres'].apply(lambda x: x.split('|'))

Group by Genre

In [81]:
movies_grouped_by_genre = movies_with_ratings_combined_df.explode('genres').groupby('genres')

Aggregrate the results

In [82]:
all_genres = movies_grouped_by_genre.agg(sum).reset_index()['genres']
all_genres

0          action
1       adventure
2       animation
3        children
4          comedy
5           crime
6     documentary
7           drama
8         fantasy
9       film-noir
10         horror
11           imax
12        musical
13        mystery
14        romance
15         sci-fi
16       thriller
17            war
18        western
Name: genres, dtype: object

create a list of genres

In [84]:
all_genres_list = list(all_genres)
all_genres_list

['action',
 'adventure',
 'animation',
 'children',
 'comedy',
 'crime',
 'documentary',
 'drama',
 'fantasy',
 'film-noir',
 'horror',
 'imax',
 'musical',
 'mystery',
 'romance',
 'sci-fi',
 'thriller',
 'war',
 'western']

Let's turn our list of genres into a space separated list of genres (as if they were words in a sentence)

In [86]:
movies_with_ratings_combined_df['genres_as_string'] = movies_with_ratings_combined_df['genres'].apply(lambda x: ' '.join(x))

reset our index again

In [87]:
movies_with_ratings_combined_df = movies_with_ratings_combined_df.reset_index().drop(columns=['index'])

In [88]:
from sklearn.feature_extraction.text import TfidfVectorizer
tf = TfidfVectorizer(analyzer='word',ngram_range=(1, 1),min_df=0, stop_words='english', vocabulary=all_genres_list)
tfidf_matrix = tf.fit_transform(movies_with_ratings_combined_df['genres_as_string'])

To see what our matrix looks like we can do the following:

In [89]:
pd.DataFrame(tfidf_matrix.toarray())

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18
0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,1.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0
1,0.0,0.0,0.000000,0.000000,0.775664,0.000000,0.0,0.631147,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0
2,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,1.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0
3,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.292236,0.0,0.0,0.529943,0.0,0.0,0.645142,0.000000,0.0,0.466423,0.000000,0.0
4,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,1.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50091,0.0,0.0,0.000000,0.000000,0.440962,0.671268,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.595778,0.0,0.000000,0.000000,0.0
50092,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.378841,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.925462,0.0
50093,0.0,0.0,0.000000,0.000000,0.486407,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.873732,0.000000,0.0,0.000000,0.000000,0.0
50094,0.0,0.0,0.707986,0.706226,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0


Now let's calculate the dot product of the tfidf_martix with itself in order to get a cosine similarity matrix.

In [90]:
from sklearn.metrics.pairwise import linear_kernel
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

 What do we expect the dimensions of this matrix to be?

In [91]:
cosine_sim

array([[1.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 1.        , 0.        , ..., 0.37728826, 0.        ,
        0.        ],
       [0.        , 0.        , 1.        , ..., 0.        , 0.        ,
        1.        ],
       ...,
       [0.        , 0.37728826, 0.        , ..., 1.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 1.        ,
        0.        ],
       [0.        , 0.        , 1.        , ..., 0.        , 0.        ,
        1.        ]])

Notice that this matrix also has 1's along the diagonals. Why is that?

In [92]:
cosine_sim.shape

(50096, 50096)

The next part you can Test Drive because now we'll add logic to grab the top 20 movie titles by index

We can build this in a separate module and import it here to see some results

In [93]:
from analysis.utils.recommendation import get_similar_movies

similar_movies = get_similar_movies('Toy Story (1995)', cosine_sim, movies_with_ratings_combined_df)
similar_movies

['Antz (1998)', 'Asterix and the Vikings (Astérix et les Vikings) (2006)']

THAT'S IT! :)